In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

In this lab, you'll be working with the Coffee Quality Index dataset, located [here](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). For convenience (and to save trouble in case you can't download files, or someone uploads a newer version), I've provided the dataset in the `data/` folder. The metadata (description) is at the Kaggle link. For this lab, you'll only need `merged_data_cleaned.csv`, as it is the concatenation of the other two datasets.

In this (and the following labs), you'll get several questions and problems. Do your analysis, describe it, use any tools and plots you wish, and answer. You can create any amount of cells you'd like.

Sometimes, the answers will not be unique, and they will depend on how you decide to approach and solve the problem. This is usual - we're doing science after all!

It's a good idea to save your clean dataset after all the work you've done to it.

### Problem 1. Read the dataset (1 point)
This should be self-explanatory. The first column is the index.

In [2]:
coffee_data = pd.read_csv("data_exercise/merged_data_cleaned.csv")
coffee_data

,Unnamed: 0,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0
3,3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,NaN,wolensu,NaN,yidnekachew debessa coffee plantation,1800-2200,...,Green,2,"March 25th, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1800.0,2200.0,2000.0
4,4,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,2,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,1334,Robusta,luis robles,Ecuador,robustasa,Lavado 1,our own lab,NaN,robustasa,NaN,...,Blue-Green,1,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN
1335,1335,Robusta,luis robles,Ecuador,robustasa,Lavado 3,own laboratory,NaN,robustasa,40,...,Blue-Green,0,"January 18th, 2017",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,40.0,40.0,40.0
1336,1336,Robusta,james moore,United States,fazenda cazengo,NaN,cafe cazengo,NaN,global opportunity fund,795 meters,...,NaN,6,"December 23rd, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,795.0,795.0,795.0
1337,1337,Robusta,cafe politico,India,NaN,NaN,NaN,14-1118-2014-0087,cafe politico,NaN,...,Green,1,"August 25th, 2015",Specialty Coffee Association,ff7c18ad303d4b603ac3f8cff7e611ffc735e720,352d0cf7f3e9be14dad7df644ad65efc27605ae2,m,NaN,NaN,NaN


### Problem 2. Observations and features (1 point)
How many observations are there? How many features? Which features are numerical, and which are categorical?

**Note:** Think about the _meaning_, not the data types. The dataset hasn't been thoroughly cleaned.

### Number of observations (rows in our dataset): 1339

### Number of features (columns in our dataset): 45

In [3]:
coffee_data.dtypes

Unnamed: 0                 int64
Species                   object
Owner                     object
Country.of.Origin         object
Farm.Name                 object
Lot.Number                object
Mill                      object
ICO.Number                object
Company                   object
Altitude                  object
Region                    object
Producer                  object
Number.of.Bags             int64
Bag.Weight                object
In.Country.Partner        object
Harvest.Year              object
Grading.Date              object
Owner.1                   object
Variety                   object
Processing.Method         object
Aroma                    float64
Flavor                   float64
Aftertaste               float64
Acidity                  float64
Body                     float64
Balance                  float64
Uniformity               float64
Clean.Cup                float64
Sweetness                float64
Cupper.Points            float64
Total.Cup.

In [4]:
print(f"Observations - {coffee_data.shape[0]}")
print(f"Features - {coffee_data.shape[1]}")

Observations - 1339
Features - 44


In [5]:
numerical_features = ['aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance',
 'uniformity', 'clean_cup', 'sweetness', 'cupper_points',
 'total_cup_points', 'moisture', 'category_one_defects',
 'category_two_defects', 'quakers', 'number_of_bags',
 'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters',
 'altitude_parsed']

categorical_features = ['unnamed:_0', 'species', 'owner', 'country_of_origin', 'farm_name', 'lot_number',
 'mill', 'ico_number', 'company', 'altitude', 'region', 'producer',
 'bag_weight', 'in_country_partner', 'harvest_year', 'grading_date',
 'owner_1', 'variety', 'processing_method', 'color', 'expiration',
 'certification_body', 'certification_address', 'certification_contact',
 'unit_of_measurement']

print(f"Numerical features count - {len(numerical_features)}")
print(f"Categorical features count - {len(categorical_features)}")

Numerical features count - 20
Categorical features count - 25


### Problem 3. Column manipulation (1 point)
Make the column names more Pythonic (which helps with the quality and... aesthetics). Convert column names to `snake_case`, i.e. `species`, `country_of_origin`, `ico_number`, etc. Try to not do it manually.

In [6]:
coffee_data.columns = (
    coffee_data.columns
    .str.strip()
    .str.lower()
    .str.replace('.', '_')  
)

coffee_data.dtypes

unnamed: 0                 int64
species                   object
owner                     object
country_of_origin         object
farm_name                 object
lot_number                object
mill                      object
ico_number                object
company                   object
altitude                  object
region                    object
producer                  object
number_of_bags             int64
bag_weight                object
in_country_partner        object
harvest_year              object
grading_date              object
owner_1                   object
variety                   object
processing_method         object
aroma                    float64
flavor                   float64
aftertaste               float64
acidity                  float64
body                     float64
balance                  float64
uniformity               float64
clean_cup                float64
sweetness                float64
cupper_points            float64
total_cup_

### Problem 4. Bag weight (1 point)
What's up with the bag weights? Make all necessary changes to the column values. Don't forget to document your methods and assumptions.

In [7]:
coffee_data.bag_weight.value_counts(dropna=False)

bag_weight
1 kg        331
60 kg       256
69 kg       200
70 kg       156
2 kg        122
100 lbs      59
30 kg        29
5 lbs        23
6            19
20 kg        14
50 kg        14
10 kg        11
59 kg        10
1 lbs         8
3 lbs         7
1             7
5 kg          7
2 lbs         5
18975 kg      4
80 kg         4
4 lbs         4
0 lbs         3
46 kg         3
6 kg          2
9000 kg       2
25 kg         2
35 kg         2
12000 kg      2
19200 kg      2
15 kg         2
40 kg         2
66 kg         2
29 kg         2
1 kg,lbs      1
2             1
2 kg,lbs      1
34 kg         1
18000 kg      1
132 lbs       1
660 kg        1
18 kg         1
150 lbs       1
1218 kg       1
0 kg          1
13800 kg      1
1500 kg       1
24 kg         1
80 lbs        1
8 kg          1
3 kg          1
350 kg        1
67 kg         1
4 kg          1
55 lbs        1
100 kg        1
130 lbs       1
Name: count, dtype: int64

In [8]:
bag_weights = coffee_data.bag_weight.str.lower().fillna('')

def classify_unit(value):
    """
    Helper function to classify a value to a given unit based on our assumptions
    """
    
    has_kg = 'kg' in value
    has_lbs = 'lbs' in value
    
    if has_kg and has_lbs:
        return 'ambiguous'
    elif has_lbs:
        return 'lbs'
    elif has_kg:
        return 'kg'
    elif value.strip().isdigit():
        return 'no_unit'
    else:
        return 'other'

unit_classifications = bag_weights.apply(classify_unit)

unit_counts = unit_classifications.value_counts()
unit_counts

bag_weight
kg           1196
lbs           114
no_unit        27
ambiguous       2
Name: count, dtype: int64

We observed that the column contains a mix of formats. We classified all values into 4 categories. We assume that values with no unit (2%) are in kilograms, because 89% of all entries are clearly labeled as "kg". This makes "kg" the dominant and default unit. We exclude the two ambiguous entries ($\sim$ 0.1%) due to unclear meaning - we make them NaN.

In [9]:
def convert_bag_weight(value):
    """
    Helper function to convert a value to kg based on its measure
    """
    
    if pd.isna(value):
        return None

    value = str(value).lower().strip()

    if 'kg' in value and 'lbs' in value:
        return None

    if 'kg' in value:
        value = value.replace('kg', '').strip()
        if value.replace('.', '', 1).isdigit():
            return float(value)

    elif 'lbs' in value:
        value = value.replace('lbs', '').strip()
        if value.replace('.', '', 1).isdigit():
            return round(float(value) * 0.453592, 2)

    elif value.replace('.', '', 1).isdigit():
        return float(value)

    return None

In [10]:
coffee_data.bag_weight = coffee_data.bag_weight.apply(convert_bag_weight)
coffee_data.dtypes

unnamed: 0                 int64
species                   object
owner                     object
country_of_origin         object
farm_name                 object
lot_number                object
mill                      object
ico_number                object
company                   object
altitude                  object
region                    object
producer                  object
number_of_bags             int64
bag_weight               float64
in_country_partner        object
harvest_year              object
grading_date              object
owner_1                   object
variety                   object
processing_method         object
aroma                    float64
flavor                   float64
aftertaste               float64
acidity                  float64
body                     float64
balance                  float64
uniformity               float64
clean_cup                float64
sweetness                float64
cupper_points            float64
total_cup_

In [11]:
coffee_data.bag_weight.value_counts(dropna=False)

bag_weight
1.00        338
60.00       256
69.00       200
70.00       156
2.00        123
45.36        59
30.00        29
2.27         23
6.00         21
20.00        14
50.00        14
10.00        11
59.00        10
0.45          8
1.36          7
5.00          7
0.91          5
0.00          4
80.00         4
1.81          4
18975.00      4
46.00         3
15.00         2
40.00         2
19200.00      2
NaN           2
66.00         2
25.00         2
9000.00       2
35.00         2
12000.00      2
29.00         2
59.87         1
34.00         1
660.00        1
18.00         1
68.04         1
18000.00      1
1218.00       1
13800.00      1
1500.00       1
24.00         1
36.29         1
8.00          1
3.00          1
350.00        1
67.00         1
4.00          1
24.95         1
100.00        1
58.97         1
Name: count, dtype: int64

In [12]:
coffee_data.bag_weight

0       60.00
1       60.00
2        1.00
3       60.00
4       60.00
        ...  
1334     2.00
1335     2.00
1336     1.00
1337     2.27
1338     2.27
Name: bag_weight, Length: 1339, dtype: float64

### Problem 5. Dates (1 point)
This should remind you of problem 4 but it's slightly nastier. Fix the harvest years, document the process.

While you're here, fix the expiration dates, and grading dates. Unlike the other column, these should be dates (`pd.to_datetime()` is your friend).

In [13]:
coffee_data.harvest_year.value_counts(dropna=False)

harvest_year
2012                        354
2014                        233
2013                        181
2015                        129
2016                        124
2017                         70
NaN                          47
2013/2014                    29
2015/2016                    28
2011                         26
2017 / 2018                  19
2014/2015                    19
2009/2010                    12
2010                         10
2010-2011                     6
2016 / 2017                   6
4T/10                         4
2009-2010                     3
March 2010                    3
Mayo a Julio                  3
4T/2010                       3
January 2011                  2
Abril - Julio                 2
08/09 crop                    2
2011/2012                     2
Sept 2009 - April 2010        1
May-August                    1
Fall 2009                     1
mmm                           1
2009 / 2010                   1
2009 - 2010                

We need to standardize the harvest_year column to contain only valid 4-digit numeric years so that the data can be used properly. There are single valid years - for example - 2012, 2015. But, there are year ranges (like 2013/2014), some strings with years ("March 2010"), some meaningless text - like "TEST". We filter the years - we take the single valid years, for the ranges - we take the first year (the first year of the harvest season), for descriptive strings (like "March 2010") - we take the year only and, finally, for meaningless text - we discard it as NaN. We also need to keep only valid year values between 1900 and 2025. (1900 - coffee data probably doesn't go earlier - here we focus on recorded harvest data - which may not go before the 20th century, maximum year is the current one - to catch future typos).

In [14]:
coffee_data.harvest_year = coffee_data.harvest_year.str.lower().str.strip().str.extract(r'(\d{4})')[0].astype('Int64')

coffee_data.harvest_year = coffee_data.harvest_year.where(
    (coffee_data.harvest_year >= 1900) & (coffee_data.harvest_year <= 2025)
)

coffee_data.harvest_year.value_counts(dropna=False)

harvest_year
2012    354
2014    252
2013    210
2015    157
2016    131
2017     89
<NA>     64
2011     36
2010     25
2009     20
2018      1
Name: count, dtype: Int64

In [15]:
coffee_data.harvest_year

0       2014
1       2014
2       <NA>
3       2014
4       2014
        ... 
1334    2016
1335    2016
1336    2014
1337    2013
1338    2013
Name: harvest_year, Length: 1339, dtype: Int64

In [16]:
coffee_data.dtypes

unnamed: 0                 int64
species                   object
owner                     object
country_of_origin         object
farm_name                 object
lot_number                object
mill                      object
ico_number                object
company                   object
altitude                  object
region                    object
producer                  object
number_of_bags             int64
bag_weight               float64
in_country_partner        object
harvest_year               Int64
grading_date              object
owner_1                   object
variety                   object
processing_method         object
aroma                    float64
flavor                   float64
aftertaste               float64
acidity                  float64
body                     float64
balance                  float64
uniformity               float64
clean_cup                float64
sweetness                float64
cupper_points            float64
total_cup_

In [17]:
coffee_data.expiration.value_counts(dropna=False)

expiration
December 26th, 2014    25
July 11th, 2013        25
June 6th, 2013         19
August 30th, 2013      18
July 26th, 2013        15
                       ..
November 30th, 2012     1
January 27th, 2015      1
June 15th, 2016         1
January 16th, 2018      1
May 14th, 2014          1
Name: count, Length: 566, dtype: int64

In [18]:
coffee_data.grading_date.value_counts(dropna=False)

grading_date
July 11th, 2012        25
December 26th, 2013    24
June 6th, 2012         19
August 30th, 2012      18
July 26th, 2012        15
                       ..
January 17th, 2011      1
August 23rd, 2016       1
August 5th, 2014        1
August 17th, 2016       1
October 25th, 2017      1
Name: count, Length: 567, dtype: int64

Here, the dates have different formats - by adding errors='coerce' and dayfirst = True to the to_datetime() function - we replace bad values with NaT and help parse formats like "31/05/2011" correctly. Before converting, we remove suffixes from the numbers (like th,nd, ...)

In [19]:
coffee_data.expiration = (
    coffee_data.expiration.astype(str)
    .str.lower()
    .str.replace(r'(\d{1,2})(st|nd|rd|th)', r'\1', regex=True)
    .str.strip()
)

In [20]:
coffee_data.expiration = pd.to_datetime(coffee_data.expiration, errors='coerce')

C:\Users\Jerry\AppData\Local\Temp\ipykernel_8436\1902029930.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  coffee_data.expiration = pd.to_datetime(coffee_data.expiration, errors='coerce')


In [21]:
coffee_data.expiration.value_counts(dropna=False)

expiration
2014-12-26    25
2013-07-11    25
2013-06-06    19
2013-08-30    18
2013-07-26    15
              ..
2015-08-05     1
2017-08-17     1
2018-10-25     1
2018-10-31     1
2013-01-04     1
Name: count, Length: 565, dtype: int64

In [22]:
coffee_data.grading_date = (
    coffee_data.grading_date.astype(str)
    .str.lower()
    .str.replace(r'(\d{1,2})(st|nd|rd|th)', r'\1', regex=True)
    .str.strip()
)

In [23]:
coffee_data.grading_date = pd.to_datetime(coffee_data.grading_date, errors='coerce')

C:\Users\Jerry\AppData\Local\Temp\ipykernel_8436\3509975755.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  coffee_data.grading_date = pd.to_datetime(coffee_data.grading_date, errors='coerce')


In [24]:
coffee_data.grading_date.value_counts(dropna=False)

grading_date
2013-12-26    25
2012-07-11    25
2012-06-06    19
2012-08-30    18
2012-07-26    15
              ..
2014-08-05     1
2016-08-17     1
2017-10-25     1
2017-10-31     1
2012-01-05     1
Name: count, Length: 565, dtype: int64

In [27]:
coffee_data.dtypes

unnamed: 0                        int64
species                          object
owner                            object
country_of_origin                object
farm_name                        object
lot_number                       object
mill                             object
ico_number                       object
company                          object
altitude                         object
region                           object
producer                         object
number_of_bags                    int64
bag_weight                      float64
in_country_partner               object
harvest_year                      Int64
grading_date             datetime64[ns]
owner_1                          object
variety                          object
processing_method                object
aroma                           float64
flavor                          float64
aftertaste                      float64
acidity                         float64
body                            float64


### Problem 6. Countries (1 point)
How many coffees are there with unknown countries of origin? What can you do about them?

In [28]:
coffee_data.country_of_origin.value_counts(dropna=False)

country_of_origin
Mexico                          236
Colombia                        183
Guatemala                       181
Brazil                          132
Taiwan                           75
United States (Hawaii)           73
Honduras                         53
Costa Rica                       51
Ethiopia                         44
Tanzania, United Republic Of     40
Uganda                           36
Thailand                         32
Nicaragua                        26
Kenya                            25
El Salvador                      21
Indonesia                        20
China                            16
India                            14
Malawi                           11
United States                    10
Peru                             10
Vietnam                           8
Myanmar                           8
Haiti                             6
Philippines                       5
United States (Puerto Rico)       4
Panama                            4
Laos      

There is only 1 coffee record with an unknown country_of_origin, which is about 0.07% of the dataset. Since this is a very small fraction, we can safely leave it as NaN, which will be ignored in most aggregations and visualizations. Alternatively, it can be labeled "Unknown" for categorical grouping.

### Problem 7. Owners (1 point)
There are two suspicious columns, named `Owner`, and `Owner.1` (they're likely called something different after you solved problem 3). Do something about them. Is there any link to `Producer`?

We have two columns that clearly hold similar company names but in different formats. So owner_1 seems to be a capitalized or formatted version of owner, but they contain the same values. We need to keep only one column, preferably the one that has better formatting (like capitalization) and which matches naming conventions in other columns. So, we drop the redundant one (likely the lowercased 'owner')

In [39]:
coffee_data.owner

0                      metad plc
1                      metad plc
2       grounds for health admin
3            yidnekachew dabessa
4                      metad plc
                  ...           
1334                 luis robles
1335                 luis robles
1336                 james moore
1337               cafe politico
1338               cafe politico
Name: owner, Length: 1339, dtype: object

In [38]:
coffee_data.owner_1

0                      metad plc
1                      metad plc
2       Grounds for Health Admin
3            Yidnekachew Dabessa
4                      metad plc
                  ...           
1334                 Luis Robles
1335                 Luis Robles
1336                 James Moore
1337               Cafe Politico
1338               Cafe Politico
Name: owner_1, Length: 1339, dtype: object

In [40]:
coffee_data = coffee_data.drop(columns=['owner'])
coffee_data = coffee_data.rename(columns={'owner_1': 'owner'})
coffee_data.owner

0                      metad plc
1                      metad plc
2       Grounds for Health Admin
3            Yidnekachew Dabessa
4                      metad plc
                  ...           
1334                 Luis Robles
1335                 Luis Robles
1336                 James Moore
1337               Cafe Politico
1338               Cafe Politico
Name: owner, Length: 1339, dtype: object

In [41]:
coffee_data.producer

0                                   METAD PLC
1                                   METAD PLC
2                                         NaN
3       Yidnekachew Dabessa Coffee Plantation
4                                   METAD PLC
                        ...                  
1334            Café Robusta del Ecuador S.A.
1335            Café Robusta del Ecuador S.A.
1336                             Cafe Cazengo
1337                                      NaN
1338                                      NaN
Name: producer, Length: 1339, dtype: object

We check if producer and owner are the same - in our case, some producers do match owners exactly. Others are different - so they're not always duplicates. So keep both owner and producer - they describe related but not identical entities.

### Problem 8. Coffee color by country and continent (1 point)
Create a table which shows how many coffees of each color are there in every country. Leave the missing values as they are.

**Note:** If you ask me, countries should be in rows, I prefer long tables much better than wide ones.

Now do the same for continents. You know what continent each country is located in.

### Problem 9. Ratings (1 point)
The columns `Aroma`, `Flavor`, etc., up to `Moisture` represent subjective ratings. Explore them. Show the means and range; draw histograms and / or boxplots as needed. You can even try correlations if you want. What's up with all those ratings?

### Problem 10. High-level errors (1 point)
Check the countries against region names, altitudes, and companies. Are there any discrepancies (e.g. human errors, like a region not matching the country)? Take a look at the (cleaned) altitudes; there has been a lot of preprocessing done to them. Was it done correctly?

### * Problem 11. Clean and explore at will
The dataset claimed to be clean, but we were able to discover a lot of things to fix and do better.

Play around with the data as much as you wish, and if you find variables to tidy up and clean - by all means, do that!